<a href="https://colab.research.google.com/github/aryan1429/deep-learning-lab/blob/main/cnn_cifar_pytorch(w5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = datasets.CIFAR10(root='./data', train=True , download = True , transform = transform)
test_data = datasets.CIFAR10(root = '/data', train = False , download = True , transform = transform)

100%|██████████| 170M/170M [00:04<00:00, 40.0MB/s]
100%|██████████| 170M/170M [00:04<00:00, 40.3MB/s]


In [5]:
train_loader = DataLoader(dataset = train_data, batch_size = 64, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = 64, shuffle = False)

In [10]:
#Build the Architecture
class cnn_cifar(nn.Module):
  def __init__(self):
    super(cnn_cifar,self).__init__()
    self.cnn1 = nn.Conv2d(3,6,5)
    self.cnn2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(5 * 5 * 16 , 120)
    self.fc2 = nn.Linear(120 , 84)
    self.fc3 = nn.Linear(84 , 10)
  def forward(self,x):
    x = self.cnn1(x)
    x = torch.relu(x)
    x = torch.max_pool2d(x,2)

    x = self.cnn2(x)
    x = torch.relu(x)
    x = torch.max_pool2d(x,2)

    #flatten
    x = x.flatten(1)

    x = self.fc1(x)
    x = torch.relu(x)

    x = self.fc2(x)
    x = torch.relu(x)

    x = self.fc3(x)

    return x

In [12]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [14]:
for epoch in range(10):
  for images, label in  train_loader:
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output,label)
    loss.backward()
    optimizer.step()
  print(f'Epoch {epoch+1} , Loss : {loss.item()}')

Epoch 1 , Loss : 1.5267173051834106
Epoch 2 , Loss : 1.7597362995147705
Epoch 3 , Loss : 1.1217737197875977
Epoch 4 , Loss : 1.192388892173767
Epoch 5 , Loss : 1.1357418298721313
Epoch 6 , Loss : 0.9621040225028992
Epoch 7 , Loss : 1.1739195585250854
Epoch 8 , Loss : 0.7139668464660645
Epoch 9 , Loss : 0.9123035073280334
Epoch 10 , Loss : 1.3379175662994385


In [17]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)

    _,predictions = torch.max(outputs.data,1)
    correct += (predictions == labels).sum().item()
    total += labels.size(0)
print(f'Accuracy {(correct / total)*100}')


Accuracy 63.470000000000006
